##  SENTIMENT ANALYSIS FOR KENYA AIRWAYS

###  AUTHORS

* Winfred Kinya
* Jeremiah Waiguru
* Paul Ngatia
* Mercy Kiragu

##  1.0  PROJECT OVERVIEW

This project is focused on analyzing customer reviews of Kenya Airways to extract meaningful insights regarding customer satisfaction and service quality. By utilizing sentiment analysis techniques, the project will classify these reviews into positive, negative, or neutral sentiments. The goal is to help Kenya Airways identify areas of excellence as well as aspects of their service that may require improvement. This analysis will serve as a foundation for enhancing customer experience and driving operational improvements.

##  1.1  BUSINESS UNDERSTANDING

In the competitive aviation industry, customer satisfaction plays a vital role in shaping the reputation and success of an airline. Kenya Airways, like many airlines, receives numerous reviews from passengers, which contain valuable feedback about their experiences. Understanding the sentiments behind these reviews is essential for identifying strengths and weaknesses in service delivery. This project will leverage sentiment analysis to process and analyze customer feedback, enabling Kenya Airways to make informed decisions that enhance passenger satisfaction and loyalty.

##  1.2  PROBLEM STATEMENT

Kenya Airways is challenged by the vast amount of unstructured customer feedback available through online reviews. Without an efficient method to analyze this feedback, the airline may overlook critical insights that could lead to service improvements. This project aims to address this challenge by systematically analyzing the sentiment of customer reviews, providing Kenya Airways with a clear understanding of customer perceptions and identifying key areas that require attention.

##  1.3  OBJECTIVES

### Primary objective

To develop a robust sentiment analysis model that accurately classifies customer reviews of Kenya Airways into positive, negative, or neutral sentiments.

###  Specific objectives

1. To explore the sentiment distribution across different customer demographics, travel classes, and time periods.
2. To identify recurring themes and specific aspects of service (e.g., customer service, punctuality, in-flight experience) that are frequently mentioned in the reviews.
3. To assess the performance of various sentiment analysis models and determine the most effective model for this dataset.
4. To generate actionable insights from the sentiment analysis that can be used by Kenya Airways to improve customer service and address common issues

##  2.0  DATA UNDERSTANDING

We did Web Scraping from the 3 websites below and merged our datasets

1. https://uk.trustpilot.com/review/www.kenya-airways.com
2. https://www.airlinequality.com/airline-reviews/kenya-airways/
3. https://www.airlineratings.com/airlines/kenya-airways

The dataset has 3065 rows and 5 columns with the following description ;

* Departures: This column indicates the departure region or type of flight, such as "Africa" or "International."
* Class: This column specifies the class of travel, such as "Economy" or "Business Class."
* Review: This column contains customer reviews of their experiences with Kenya Airways. The text data may include opinions, complaints, and compliments about the airline.
* Rating: This column contains the numeric rating provided by the customer, typically ranging from 1.0 to 5.0, where 1.0 represents "Poor" and 5.0 represents "Excellent."
* Rating_description: This column provides a textual description corresponding to the numeric rating, such as "Poor," "Good," or "Excellent."

## Importing Necessary Libraries

In [603]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
import seaborn as sns
import plotly.express as px 
import re
import string
import joblib

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud 
from sklearn.metrics import roc_curve, auc

#downloading dependencies
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('vader_lexicon')

from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## 2. 1 Loading Datasets

### 1.Dataset one

In [604]:
#Kenya_airways_flights
df = pd.read_csv('Refined data/Dataset_one.csv')
df.head()

,Date3,Departures,Class,Review,Rating_Value,Rating_Description
0,2022-09-01,Africa,Economy,Recently I travelled for business from Cape To...,4.0,Good
1,2022-11-01,International,Economy,I want to thanks to Jane for her excellent ser...,5.0,Excellent
2,2022-05-01,International,Economy,not a bad airline to fly with could do with a ...,4.0,Good
3,2022-11-01,Africa,Business Class,"Dear Kenya Airways ,\r\n\r\nI am on my way to ...",1.0,Poor
4,2022-11-01,Africa,Economy,"Absolutely appalling airline, never ever use i...",1.0,Poor


In [605]:
def describe_columns(df):
    # Print column names
    print("Column Names:")
    print(df.columns)

    # Print number of rows and columns
    print("\nShape:")
    print(df.shape)

describe_columns(df)

Column Names:
Index(['Date3', 'Departures', 'Class', 'Review', 'Rating_Value',
       'Rating_Description'],
      dtype='object')

Shape:
(2345, 6)


The Dataset has 2345 rows and 6 columns.
The other columns are categorical or text-based.

### 2.Dataset two

In [606]:
#Kenya_airways_reviews
df = pd.read_csv('Refined data/Dataset_two.csv')
df.head()

,Date3,Review
0,2024-02-24,✅ Trip Verified | I have never seen such a dis...
1,2024-02-24,Not Verified | Bumped off flight at boarding...
2,2024-07-30,✅ Trip Verified | I am disappointment with t...
3,2024-07-25,✅ Trip Verified | This airline is becoming mo...
4,2024-07-24,✅ Trip Verified | 3 out of 4 flights delayed ...


In [607]:
def describe_columns(df):
    # Print column names
    print("Column Names:")
    print(df.columns)

    # Print number of rows and columns 
    print("\nShape:")
    print(df.shape)

describe_columns(df)

Column Names:
Index(['Date3', 'Review'], dtype='object')

Shape:
(477, 2)


The Dataset has 477 rows and 2 columns.

### 3.Dataset three

In [608]:
# KQ_reviews
df = pd.read_csv('Refined data/Dataset_three.csv')
df.head()

,Date3,Rating,Review
0,02-08-2024,0.0,Constantly canceling flights Kenya airlines ca...
1,04-08-2024,0.0,Kenya Airways canceled the flight and… Kenya A...
2,30-07-2024,0.0,If your thinking of Kenya Airlines...think aga...
3,29-07-2024,0.0,"Worst airlines ever Worst airlines ever, not f..."
4,25-07-2024,0.0,We were trying to go to Nampula from Nairobi a...


In [609]:
def describe_columns(df):
    # Print column names
    print("Column Names:")
    print(df.columns)

    # Print number of rows and columns 
    print("\nShape:")
    print(df.shape)

describe_columns(df)

Column Names:
Index(['Date3', 'Rating', 'Review'], dtype='object')

Shape:
(243, 3)


The Dataset has 243 rows and 3 columns.Mainly, 'Date', 'Rating and 'Review'

## Loading the Merged Dataset

In [610]:
# KQ_Data
df = pd.read_csv('Refined data/KQ_Data.csv')
df

,Date3,Departures,Class,Review,Rating,Rating_Description
0,2022-09-01 00:00:00,Africa,Economy,Recently I travelled for business from Cape To...,4.0,Good
1,2022-11-01 00:00:00,International,Economy,I want to thanks to Jane for her excellent ser...,5.0,Excellent
2,2022-05-01 00:00:00,International,Economy,not a bad airline to fly with could do with a ...,4.0,Good
3,2022-11-01 00:00:00,Africa,Business Class,"Dear Kenya Airways ,\r\n\r\nI am on my way to ...",1.0,Poor
4,2022-11-01 00:00:00,Africa,Economy,"Absolutely appalling airline, never ever use i...",1.0,Poor
...,...,...,...,...,...,...
3060,26-01-2017,NaN,NaN,Better than I expected Much better service tha...,0.0,NaN
3061,30-08-2016,NaN,NaN,Request for immediate action. I have never had...,0.0,NaN
3062,02-07-2016,NaN,NaN,Never again Flew with Kenya airways last year ...,0.0,NaN
3063,21-11-2015,NaN,NaN,NEVER AGAIN - RUBISH COMPANY !!! We travelled ...,1.0,NaN


In [611]:
def describe_columns(df):
    # Print column names
    print("Column Names:")
    print(df.columns)

    # Print data types
    print("\nData Types:")
    print(df.dtypes)

    # Print number of rows and columns
    print("\nShape:")
    print(df.shape)

    # Print df information
    print("\nInfo:")
    print(df.info())

    # Print descriptive statistics for numerical columns
    print("\nDescriptive Statistics:")
    print(df.describe())

    # Print missing values count per column
    print("\nMissing Values in percentages:")
    print((df.isna().sum()/len(df)) * 100)

describe_columns(df)

Column Names:
Index(['Date3', 'Departures', 'Class', 'Review', 'Rating',
       'Rating_Description'],
      dtype='object')

Data Types:
Date3                  object
Departures             object
Class                  object
Review                 object
Rating                float64
Rating_Description     object
dtype: object

Shape:
(3065, 6)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3065 entries, 0 to 3064
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date3               3063 non-null   object 
 1   Departures          2345 non-null   object 
 2   Class               2345 non-null   object 
 3   Review              3065 non-null   object 
 4   Rating              2578 non-null   float64
 5   Rating_Description  2345 non-null   object 
dtypes: float64(1), object(5)
memory usage: 143.8+ KB
None

Descriptive Statistics:
            Rating
count  2578.000000
mean      2.865012
std 

The dataset has 3065 rows and 6 columns. It also has missing values, with some columns registering aproximately 90%. This will be dealt with at the preprocessing stage. From our data overview, the Review column needs preprocessing since it contains raw text with numbers, nametags,emojis and punctuations.

## 3.0 DATA PREPARATION

### 3.1  Data Cleaning

Steps to start cleaning the data:

*  Focus on the columns or attributes that are relevant for your analysis or purposes.

* Handle Missing Values: Check for missing values in the dataset and decide how to handle them. You can either remove rows with missing values, replace them with appropriate values (e.g., mean, median, mode), or use advanced techniques like imputation based on the available data.

* Standardize Formats: Check for inconsistencies in the data formats. For example, ensure that dates, addresses, or numerical values are consistently formatted across the dataset.

* Text Cleaning: If you have text data, apply text cleaning techniques to remove unwanted characters, symbols, or special characters. This can include removing punctuation, converting text to lowercase, removing stopwords, and handling any language-specific nuances.

* Validate Data Integrity: Perform a final check to ensure the integrity of the cleaned data. Validate that all required fields are populated correctly, and the data is in the expected format.

In [612]:
# Checking for missing values in each column
missing_values = df.isnull().sum()

# Display the missing values count
print("Missing values in each column:")
print(missing_values)

# Check the percentage of missing values in each column (optional)
missing_percentage = (df.isnull().sum() / len(df)) * 100
print("\nPercentage of missing values in each column:")
print(missing_percentage)


Missing values in each column:
Date3                   2
Departures            720
Class                 720
Review                  0
Rating                487
Rating_Description    720
dtype: int64

Percentage of missing values in each column:
Date3                  0.065253
Departures            23.491028
Class                 23.491028
Review                 0.000000
Rating                15.889070
Rating_Description    23.491028
dtype: float64


Overall Insights:
* High Missing Data: Columns Departures, Class, and Rating_Description have a significant amount of missing data (23.49%). 
* Depending on the analysis, we have the option to either impute these values, drop the rows/columns, or find another way to handle them.(Use placeholders)
* Moderate Missing Data: The Rating column has a moderate level of missing values (15.89%), which could influence the analysis if not addressed.
* Low Missing Data: The Date3 column has very few missing values (0.07%), making it easier to handle.

###  dealing with missing values
For the columns: (Departures, Class, Date3, and Rating_Description) we will replace them with placeholders so as to retain them since they can be helpful in analysis. For the Rating column however, we will fill missing values with the median of that column.

In [613]:
df = pd.read_csv('Refined data/KQ_Data.csv')

# Replace missing values with placeholders
# Specify the placeholder for each column or use a general placeholder for all missing values

df['Date3'].fillna('Missing Date', inplace=True)
df['Departures'].fillna('Unknown', inplace=True)
df['Class'].fillna('Unknown', inplace=True)
df['Rating'] = df['Rating'].fillna(df['Rating'].median())
df['Rating_Description'].fillna('Not Rated', inplace=True)

# Check the result
df.head(5)


,Date3,Departures,Class,Review,Rating,Rating_Description
0,2022-09-01 00:00:00,Africa,Economy,Recently I travelled for business from Cape To...,4.0,Good
1,2022-11-01 00:00:00,International,Economy,I want to thanks to Jane for her excellent ser...,5.0,Excellent
2,2022-05-01 00:00:00,International,Economy,not a bad airline to fly with could do with a ...,4.0,Good
3,2022-11-01 00:00:00,Africa,Business Class,"Dear Kenya Airways ,\r\n\r\nI am on my way to ...",1.0,Poor
4,2022-11-01 00:00:00,Africa,Economy,"Absolutely appalling airline, never ever use i...",1.0,Poor


In [614]:
# Checking for missing values in each column
missing_values = df.isnull().sum()
missing_values

Date3                 0
Departures            0
Class                 0
Review                0
Rating                0
Rating_Description    0
dtype: int64

Our Dataset is now okay and does not contain any Missing data

## Data Preprocessing

#### Feature selection

In [615]:
df.columns

Index(['Date3', 'Departures', 'Class', 'Review', 'Rating',
       'Rating_Description'],
      dtype='object')

In [616]:
KQ_df = df[['Rating_Description', 'Review']]
KQ_df.head(10)

,Rating_Description,Review
0,Good,Recently I travelled for business from Cape To...
1,Excellent,I want to thanks to Jane for her excellent ser...
2,Good,not a bad airline to fly with could do with a ...
3,Poor,"Dear Kenya Airways ,\r\n\r\nI am on my way to ..."
4,Poor,"Absolutely appalling airline, never ever use i..."
5,Excellent,We flew from Heathrow to Nairobi on our way to...
6,Poor,Was travling from Kenya to South Africa last w...
7,Good,"After reading the reviews on here, I was prepa..."
8,Poor,"Kenya Airways is very disorganzied, first of a..."
9,Poor,I travelled to London on July 10th ON KQ100 wi...


From our data overview, the Review column contains raw text with unwanted characters, numbers,whitespaces, punctuations and also nametags which will need to be parsed.

In [617]:
# Remove special characters, unwanted characters, unnecessary phrases
def remove_special_characters(text):
    text = re.sub(r'\r\n|\n', ' ', text)
    return text

def remove_extra_whitespace(text):
    text = ' '.join(text.split())
    return text

def convert_to_lowercase(text):
    text = text.lower()
    return text

def remove_unnecessary_phrases(text):
    text = re.sub(r'dear\s+[\w\s]+,', '', text)
    return text

def standardize_rating_descriptions(rating):
    rating_mapping = {
        'Excellent': 'excellent',
        'Good': 'good',
        'Poor': 'poor'
    }
    return rating_mapping.get(rating, rating)


In [618]:
def clean_text(text):
    text = remove_special_characters(text)
    text = remove_extra_whitespace(text)
    text = convert_to_lowercase(text)
    text = remove_unnecessary_phrases(text)
    return text

# Apply the cleaning function to the 'Review' column
KQ_df['Review'] = KQ_df['Review'].apply(clean_text)

# Apply the standardization function to the 'Rating_Description' column
KQ_df['Rating_Description'] = KQ_df['Rating_Description'].apply(standardize_rating_descriptions)

# Display the cleaned DataFrame
KQ_df.head(10)


,Rating_Description,Review
0,good,recently i travelled for business from cape to...
1,excellent,i want to thanks to jane for her excellent ser...
2,good,not a bad airline to fly with could do with a ...
3,poor,"i am on my way to zanzibar from bangui, with ..."
4,poor,"absolutely appalling airline, never ever use i..."
5,excellent,we flew from heathrow to nairobi on our way to...
6,poor,was travling from kenya to south africa last w...
7,good,"after reading the reviews on here, i was prepa..."
8,poor,"kenya airways is very disorganzied, first of a..."
9,poor,i travelled to london on july 10th on kq100 wi...


In [619]:
def remove_nametags(sentence):
    """A simple function to remove name tags"""
    clean_words = []
    for word in sentence.split():
        if "@" not in word:
            clean_words.append(word)
    return " ".join(clean_words)

KQ_df["Review"] = KQ_df["Review"].apply(remove_nametags)
KQ_df.head(10)

,Rating_Description,Review
0,good,recently i travelled for business from cape to...
1,excellent,i want to thanks to jane for her excellent ser...
2,good,not a bad airline to fly with could do with a ...
3,poor,"i am on my way to zanzibar from bangui, with a..."
4,poor,"absolutely appalling airline, never ever use i..."
5,excellent,we flew from heathrow to nairobi on our way to...
6,poor,was travling from kenya to south africa last w...
7,good,"after reading the reviews on here, i was prepa..."
8,poor,"kenya airways is very disorganzied, first of a..."
9,poor,i travelled to london on july 10th on kq100 wi...


In [620]:
# Remove numbers and punctuations
# numbers
num_pattern = r'[0-9]'
KQ_df['Review'] = KQ_df['Review'].replace(to_replace=num_pattern, value='', regex=True)

# punctuations
KQ_df['Review'] = KQ_df['Review'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

KQ_df.head(10)

,Rating_Description,Review
0,good,recently i travelled for business from cape to...
1,excellent,i want to thanks to jane for her excellent ser...
2,good,not a bad airline to fly with could do with a ...
3,poor,i am on my way to zanzibar from bangui with an...
4,poor,absolutely appalling airline never ever use it...
5,excellent,we flew from heathrow to nairobi on our way to...
6,poor,was travling from kenya to south africa last w...
7,good,after reading the reviews on here i was prepar...
8,poor,kenya airways is very disorganzied first of al...
9,poor,i travelled to london on july th on kq with c...


In [621]:
#remove emojis
def remove_emojis(data):
    """A simple function to remove all emojis"""
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"
                           u"\U00002500-\U00002BEF"
                           u"\U00002702-\U000027B0"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f926-\U0001f937"
                           u"\U00010000-\U0010ffff"
                           u"\u2640-\u2642" 
                           u"\u2600-\u2B55"
                           u"\u200d"
                           u"\u23cf"
                           u"\u23e9"
                           u"\u231a"
                           u"\ufe0f"
                           u"\u3030"
                           "]+", re.UNICODE)
    return re.sub(emoji_pattern, "", data)

KQ_df["Review"] = KQ_df["Review"].apply(remove_emojis)


In [622]:
def remove_non_standard_char(review):
    """A simple function to remove characters outside the ASCII range"""
    pattern = re.compile(r"[^\x00-\x7F]+")
    clean_string = re.sub(pattern, "", review)
    return clean_string 

KQ_df["Review"] = KQ_df["Review"].apply(remove_non_standard_char)

KQ_df.head(10)

,Rating_Description,Review
0,good,recently i travelled for business from cape to...
1,excellent,i want to thanks to jane for her excellent ser...
2,good,not a bad airline to fly with could do with a ...
3,poor,i am on my way to zanzibar from bangui with an...
4,poor,absolutely appalling airline never ever use it...
5,excellent,we flew from heathrow to nairobi on our way to...
6,poor,was travling from kenya to south africa last w...
7,good,after reading the reviews on here i was prepar...
8,poor,kenya airways is very disorganzied first of al...
9,poor,i travelled to london on july th on kq with c...


In [623]:
# remove http tags
urlpattern = r'(?:http)s?\S+'
KQ_df['Review'] = KQ_df['Review'].str.replace(urlpattern, '')

In [624]:
# list stopwords
stop_words = set(stopwords.words("english"))
include_stopwords = ["wouldn", "won", "weren", "wasn", "shouldn", "shan", "needn", "mustn",
                     "mightn", "isn", "haven", "hasn", "hadn", "doesn", "didn", "couldn",
                     "aren", "ain", "don", "not"]
stop_words = list(stop_words - set(include_stopwords)) + ['would','youve']

In [625]:
def remove_stopwords(words):
    """A simple function to remove stopwords from a string"""
    clean_words = []
    for word in words.split():
        if word not in stop_words:
            clean_words.append(word)
    return " ".join(clean_words)

KQ_df['Review'] = KQ_df['Review'].apply(remove_stopwords)
# Print the first 5 rows of the new column
KQ_df.head(10)

,Rating_Description,Review
0,good,recently travelled business cape town entebbe ...
1,excellent,want thanks jane excellent service flight conn...
2,good,not bad airline fly could better selection mov...
3,poor,way zanzibar bangui upgraded flight sky priori...
4,poor,absolutely appalling airline never ever use in...
5,excellent,flew heathrow nairobi way zanzibar plane boein...
6,poor,travling kenya south africa last week heading ...
7,good,reading reviews prepared worst say website dif...
8,poor,kenya airways disorganzied first never answer ...
9,poor,travelled london july th kq children paid upgr...


Stopwords are common in texts but offer minimal semantic value. Removing them helps reduce noise and highlights more significant words, enhancing the efficiency and effectiveness of our models. We tailored our stopword list to include terms that we believe diminish the overall sentiment.

#### Stemming

In [626]:
stemmer = SnowballStemmer("english")
#lemmatizer = WordNetLemmatizer()
def stem_words(review):
    """A function to stem words in a review text"""
    stem_words = []
    for word in review.split():
        stem_word = stemmer.stem(word)
        stem_words.append(stem_word)
    return " ".join(stem_words)

KQ_df["Review"] = KQ_df["Review"].apply(stem_words)
KQ_df.head(10)

,Rating_Description,Review
0,good,recent travel busi cape town entebb uganda lay...
1,excellent,want thank jane excel servic flight connect ke...
2,good,not bad airlin fli could better select movi mu...
3,poor,way zanzibar bangui upgrad flight sky prioriti...
4,poor,absolut appal airlin never ever use incred unr...
5,excellent,flew heathrow nairobi way zanzibar plane boe d...
6,poor,travl kenya south africa last week head board ...
7,good,read review prepar worst say websit difficult ...
8,poor,kenya airway disorganzi first never answer pho...
9,poor,travel london juli th kq children paid upgrad ...


In [627]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

def tokenize_reviews(df):
    """Function to tokenize each review in the 'Review' column."""
    df['Tokenized_Review'] = df['Review'].apply(nltk.word_tokenize)
    return df

# Apply the function to your DataFrame
KQ_df = tokenize_reviews(KQ_df)

# Display the DataFrame to see the tokenized reviews
(KQ_df[['Rating_Description','Review', 'Tokenized_Review']].head())


,Rating_Description,Review,Tokenized_Review
0,good,recent travel busi cape town entebb uganda lay...,"[recent, travel, busi, cape, town, entebb, uga..."
1,excellent,want thank jane excel servic flight connect ke...,"[want, thank, jane, excel, servic, flight, con..."
2,good,not bad airlin fli could better select movi mu...,"[not, bad, airlin, fli, could, better, select,..."
3,poor,way zanzibar bangui upgrad flight sky prioriti...,"[way, zanzibar, bangui, upgrad, flight, sky, p..."
4,poor,absolut appal airlin never ever use incred unr...,"[absolut, appal, airlin, never, ever, use, inc..."


In [628]:

# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Tokenize each review in the 'Review' column and get the impactful words
num_words = 5  # Number of impactful words to extract
KQ_df['Tokenized_Review'] = KQ_df['Review'].apply(nltk.word_tokenize)

# Calculate sentiment scores and extract impactful words
KQ_df['Impactful_Words'] = KQ_df['Tokenized_Review'].apply(
    lambda tokens: [word for word, score in sorted(
        {token: sia.polarity_scores(token)["compound"] for token in tokens}.items(),
        key=lambda x: x[1], reverse=True
    )][:num_words]
)

# Display the DataFrame with the tokenized and impactful words
(KQ_df[['Rating_Description','Review', 'Tokenized_Review', 'Impactful_Words']].head())


,Rating_Description,Review,Tokenized_Review,Impactful_Words
0,good,recent travel busi cape town entebb uganda lay...,"[recent, travel, busi, cape, town, entebb, uga...","[free, friend, enjoy, good, better]"
1,excellent,want thank jane excel servic flight connect ke...,"[want, thank, jane, excel, servic, flight, con...","[excel, nice, thank, want, jane]"
2,good,not bad airlin fli could better select movi mu...,"[not, bad, airlin, fli, could, better, select,...","[better, good, not, airlin, fli]"
3,poor,way zanzibar bangui upgrad flight sky prioriti...,"[way, zanzibar, bangui, upgrad, flight, sky, p...","[nice, support, holiday, like, yep]"
4,poor,absolut appal airlin never ever use incred unr...,"[absolut, appal, airlin, never, ever, use, inc...","[like, absolut, appal, airlin, never]"


In [629]:

# Initialize empty lists to store positive and negative impactful words
positive_impactful_words = []
negative_impactful_words = []

# Loop through each tokenized review
for tokens in KQ_df['Tokenized_Review']:
    # Calculate sentiment scores for each token
    scores = {token: sia.polarity_scores(token)["compound"] for token in tokens}
    
    # Sort tokens by their sentiment scores
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    
    # Categorize words as positive or negative based on their sentiment score
    for word, score in sorted_words:
        if score > 0:  # Positive score
            positive_impactful_words.append(word)
        elif score < 0:  # Negative score
            negative_impactful_words.append(word)

# Output the lists of positive and negative impactful words
print("Positive Impactful Words:", positive_impactful_words)
print("Negative Impactful Words:", negative_impactful_words)


Positive Impactful Words: ['free', 'friend', 'enjoy', 'good', 'better', 'help', 'comfort', 'entertain', 'sure', 'excel', 'nice', 'thank', 'want', 'better', 'good', 'nice', 'support', 'holiday', 'like', 'yep', 'reach', 'like', 'friend', 'good', 'better', 'recommend', 'entertain', 'clear', 'well', 'good', 'fine', 'recommend', 'well', 'number', 'support', 'comfort', 'strong', 'trust', 'like', 'recommend', 'great', 'friend', 'fit', 'recommend', 'free', 'enjoy', 'proud', 'better', 'hope', 'thank', 'better', 'great', 'friend', 'better', 'boost', 'accept', 'like', 'sure', 'well', 'top', 'number', 'pleasant', 'good', 'treat', 'interest', 'comfort', 'sure', 'entertain', 'warm', 'like', 'luck', 'good', 'better', 'holiday', 'clean', 'help', 'wish', 'clear', 'smile', 'entertain', 'well', 'straight', 'worth', 'want', 'care', 'care', 'better', 'pride', 'true', 'comfort', 'ok', 'want', 'good', 'pride', 'best', 'thank', 'better', 'treat', 'comfort', 'sure', 'want', 'respect', 'like', 'entertain', 'kin

In [630]:
# Dividing the KQ_df DataFrame based on the 'Rating_Description' column

# DataFrame for "Poor" ratings
neg_df = KQ_df[KQ_df['Rating_Description'] == 'poor']

# DataFrame for "Excellent" ratings
pos_df = KQ_df[KQ_df['Rating_Description'] == 'excellent']

# DataFrame for "Good" ratings
neu_df = KQ_df[KQ_df['Rating_Description'] == 'good']

# Optional: Display the first few rows of each DataFrame to verify the split
print("Negative Reviews (poor):")
(neg_df.head(), "\n")

print("Positive Reviews (excellent):")
(pos_df.head(), "\n")

print("Neutral Reviews (good):")
(neu_df.head())


Negative Reviews (poor):
Positive Reviews (excellent):
Neutral Reviews (good):


,Rating_Description,Review,Tokenized_Review,Impactful_Words
0,good,recent travel busi cape town entebb uganda lay...,"[recent, travel, busi, cape, town, entebb, uga...","[free, friend, enjoy, good, better]"
2,good,not bad airlin fli could better select movi mu...,"[not, bad, airlin, fli, could, better, select,...","[better, good, not, airlin, fli]"
7,good,read review prepar worst say websit difficult ...,"[read, review, prepar, worst, say, websit, dif...","[good, fine, read, review, prepar]"
38,good,checkin klm sky prioriti check counter subsequ...,"[checkin, klm, sky, prioriti, check, counter, ...","[love, greatest, great, super, superior]"
43,good,st famili holiday holiday us last year due cov...,"[st, famili, holiday, holiday, us, last, year,...","[love, great, wow, kind, cheer]"


In [631]:
# Ensure that the functions 'positive_impactful_words' and 'negative_impactful_words' are defined correctly
# For example:

def positive_impactful_words(review, num_words=2):
    """Gets the most impactful positive words in a review."""
    tokens = nltk.word_tokenize(review)
    sia = SentimentIntensityAnalyzer()
    scores = {token: sia.polarity_scores(token)["compound"] for token in tokens}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    impactful_words = [word for word, score in sorted_words if score > 0][:num_words]
    return impactful_words

def negative_impactful_words(review, num_words=2):
    """Gets the most impactful negative words in a review."""
    tokens = nltk.word_tokenize(review)
    sia = SentimentIntensityAnalyzer()
    scores = {token: sia.polarity_scores(token)["compound"] for token in tokens}
    sorted_words = sorted(scores.items(), key=lambda x: x[1])
    impactful_words = [word for word, score in sorted_words if score < 0][:num_words]
    return impactful_words

# Apply these functions to your DataFrame
pos_df['impactful_words'] = pos_df['Review'].apply(positive_impactful_words)
neg_df['impactful_words'] = neg_df['Review'].apply(negative_impactful_words)


In [632]:
pos_df.head()

,Rating_Description,Review,Tokenized_Review,Impactful_Words,impactful_words
1,excellent,want thank jane excel servic flight connect ke...,"[want, thank, jane, excel, servic, flight, con...","[excel, nice, thank, want, jane]","[excel, nice]"
5,excellent,flew heathrow nairobi way zanzibar plane boe d...,"[flew, heathrow, nairobi, way, zanzibar, plane...","[friend, good, better, recommend, entertain]","[friend, good]"
11,excellent,great servic friend staff belinda calvin frien...,"[great, servic, friend, staff, belinda, calvin...","[great, friend, fit, recommend, servic]","[great, friend]"
15,excellent,pleasant surpris airlin servic good smooth lan...,"[pleasant, surpris, airlin, servic, good, smoo...","[pleasant, good, surpris, airlin, servic]","[pleasant, good]"
39,excellent,amaz servic st leg room suffici that realli co...,"[amaz, servic, st, leg, room, suffici, that, r...","[great, friend, amaz, servic, st]","[great, friend]"


In [633]:
neg_df.head()

,Rating_Description,Review,Tokenized_Review,Impactful_Words,impactful_words
3,poor,way zanzibar bangui upgrad flight sky prioriti...,"[way, zanzibar, bangui, upgrad, flight, sky, p...","[nice, support, holiday, like, yep]","[bad, lost]"
4,poor,absolut appal airlin never ever use incred unr...,"[absolut, appal, airlin, never, ever, use, inc...","[like, absolut, appal, airlin, never]","[bother, avoid]"
6,poor,travl kenya south africa last week head board ...,"[travl, kenya, south, africa, last, week, head...","[clear, well, travl, kenya, south]","[complain, miss]"
8,poor,kenya airway disorganzi first never answer pho...,"[kenya, airway, disorganzi, first, never, answ...","[recommend, well, number, kenya, airway]","[bad, rude]"
9,poor,travel london juli th kq children paid upgrad ...,"[travel, london, juli, th, kq, children, paid,...","[support, comfort, travel, london, juli]",[complaint]
